In [12]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from pymodbus.client import ModbusTcpClient

In [13]:
cam = 2
#fps = 240 #Maximum achivable was 163.06

model_version = "my_model_22.0.keras"
#max_frames = 10  # Number of frames to evaluate

#image_path = "no_bottle.jpg"
#frame_buffer = []
#frame_queue = queue.Queue(maxsize=10)

Im_Width = 128
Im_Height = 128

#crop
# x = 120
# y = 0
# w = 400
# h = 200

#max_brightness = 180

#stop_event = threading.Event()

coil_address = 1542 #ModBus Y6
coil_address_write = 1540
PLC_IP = "192.168.1.111"  #PLC's IP address
PLC_PORT = 502           # Default Modbus TCP port

In [14]:
def modbus(PLC_IP, PLC_PORT):

    # Create Modbus client
    client = ModbusTcpClient(PLC_IP, port=PLC_PORT)

    # Connect to the PLC
    if client.connect():
        print("Connection successful")
    else:
        print("Failed to connect to PLC")
    return client

In [15]:
def read_PLC_output(client, coil_address):
    read_result = client.read_coils(address=coil_address)  # Read a single coil (output)
    if read_result.isError():
        print("Read Error:", read_result)
        return None
    return read_result.bits[0]  # Return the first coil state (True for ON, False for OFF)

In [16]:
def write_PLC_output(client, coil_address_write,value):
    write_result = client.write_coil(address=coil_address_write, value=value)  # Write 'True' to turn ON
    if write_result.isError():
        print("Write Error:", write_result)
    else:
        print(f"Output Y4 turned ON (Coil {coil_address_write})")

In [17]:
def process_frame(frame, target_size):

    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [18]:
def predict_frame(client, frame, model, target_size):

    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

   # Make a prediction
    prediction = model.predict(frame_array)[0][0]  # Get the confidence score
    print(f"Raw Prediction: {prediction}")

    # Interpret the prediction
    if prediction > 0.5:
        write_PLC_output(client, coil_address_write ,False)
        return "Non-Defective", prediction      
        
    else:
        write_PLC_output(client, coil_address_write,True)
        return "Defective", prediction

In [19]:
def capture(client, model_version, target_size=(Im_Width, Im_Height)):
    cap = cv2.VideoCapture(2)
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return

     # Load the model
    print("Loading model...") 
    model = load_model(model_version)
    print("Model loaded successfully.")
    
    output_dir = "frames"
    os.makedirs(output_dir, exist_ok=True)  # Ensure output directory exists
    frame_count = 0
    flag = False  # Tracks the previous state of the output

    while True:
        result = client.read_coils(1542)  # Ensure count=1 to avoid errors
        if result.isError():
            print("Modbus Read Error:", result)
            continue  # Skip iteration on error

        current_state = result.bits[0]  # Read the coil status
        print("Output status:", current_state)

        # Detect rising edge: transition from False -> True
        if current_state and not flag:
            print("TRUE - Capturing Frame")
            
            ret, frame = cap.read()
            if not ret:
                print("Error: Frame capture failed.")
                break
            
            # Save the frame
            frame_filename = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            print(f"Saved: {frame_filename}") 
            frame_count += 1

            feedback, confidence = predict_frame(client, frame, model, target_size=(Im_Width, Im_Height))

            prob = abs(round(confidence,2)- 0.5)*200
            print(f"Prediction: {feedback} (Confidence: {prob:.2f})")

            # Display feedback on the frame
            cv2.putText(frame, f"Prediction: {feedback}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Confidence: {prob:.0f}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Display the best frame
            cv2.imshow("Best Frame", frame)

        # Update flag to track previous state
        flag = current_state

        # Break on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()


In [21]:
if __name__ == "__main__":

    client = modbus(PLC_IP, PLC_PORT)
    capture(client, model_version, target_size=(Im_Width, Im_Height))


Connection successful
Loading model...
Model loaded successfully.
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output status: False
Output sta